[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel/e2eAIOK/blob/main/demo/ma/domain_adapter/Model_Adapter_Domain_Adapter_builtin_Unet_KITS19.ipynb)

# Model Adapter Domain Adapter builtin Unet KITS19

In this demo, we will introduce how to use Domain Adapter to transfer knowledge in medical image semantic segmentation.

# Content

* [Overview](#overview)
    * [Model Adapter Domain Adapter Overview](#Model-Adapter-Domain-Adapter-Overview)
* [Getting Started](#Getting-Started)
    * [1. Environment Setup](#1.-Environment-Setup)
    * [2. Workflow Prepare](#2.-Workflow-Prepare)
    * [3. Data Prepare](#3-data-prepare)
    * [4. Train](#4-train)
    * [5. Inference](#5-inference)

# Overview

## Model Adapter Domain Adapter Overview

Model Adapter is a convenient framework can be used to reduce training and inference time, or data labeling cost by efficiently utilizing public advanced models and those datasets from many domains. It mainly contains three components served for different cases: Finetuner, Distiller, and Domain Adapter. 

Directly applying pre-trained model into target domain cannot always work due to covariate shift and label shift, while fine-tuning is also not working due to the expensive labeling in some domains. Even if users invest resource in labeling, it will be time-consuming and delays the model deployment.

Domain Adapter aims at reusing the transferable knowledge with the help of another labeled dataset with same learning task. That is, achieving better generalization with little labeled target dataset or achieving a competitive performance in label-free target dataset.

The following picture show the network strcture of domain adaption, which add a discriminator to users' base network, and try to differentiate the souce domain data and target domain data, hence, it can force the feature extractor to learn a generalized feature representation among domains.

![Adapter](../imgs/adapter.png)

# Getting Started

## 1. Environment Setup

### (Option 1) Use Pip install - recommend

In [ ]:
!pip install e2eAIOK-ModelAdapter --pre

### (Option 2) Use Docker

Step1. prepare code
   ``` bash
   git clone https://github.com/intel/e2eAIOK.git
   cd e2eAIOK
   git submodule update --init –recursive
   ```
    
Step2. build docker image
   ``` bash
   python3 scripts/start_e2eaiok_docker.py -b pytorch112 --dataset_path ${dataset_path} -w ${host0} ${host1} ${host2} ${host3} --proxy  "http://addr:ip"
   ```
   
Step3. run docker and start conda env
   ``` bash
   sshpass -p docker ssh ${host0} -p 12347
   conda activate pytorch-1.12.0
   ```
  
Step4. Start the jupyter notebook and tensorboard service
   ``` bash
   nohup jupyter notebook --notebook-dir=/home/vmagent/app/e2eaiok --ip=${hostname} --port=8899 --allow-root &
   nohup tensorboard --logdir /home/vmagent/app/data/tensorboard --host=${hostname} --port=6006 & 
   ```
   Now you can visit demso in `http://${hostname}:8899/`, and see tensorboad log in ` http://${hostname}:6006`.

## 2. Workflow Prepare

- First Clone the demo code
    ``` bash
    git clone https://github.com/intel/e2eAIOK.git
    cd e2eAIOK
    git submodule update --init –recursive
    ```
- Then add a patch to that code to get the full demo code
    ```bash
    cd modelzoo/unet && sh patch_unet.sh 
    ```

## 3. Data Prepare

* Our source domain is AMOS dataset(Download AMOS data from [here](https://amos22.grand-challenge.org/Dataset/)), which provides 500 CT and 100 MRI scans with voxel-level annotations of 15 abdominal organs, including the spleen, right kidney, left kidney, gallbladder, esophagus, liver, stomach, aorta, inferior vena cava, pancreas, right adrenal gland, left adrenal gland, duodenum, bladder, prostate/uterus.
* Our target domain is KiTS dataset(Download KiTS data from [here](https://github.com/neheller/kits19)), which provides 300 CT scans with voxel-level annotations of kidney organs and kidney tumor.
* After downloading the code, remember to put all your data in right places, now your files should be located at:
   - Images at: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/imagesTr/```
   - Labels/Segmentations at: ```${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/labelsTr/```
   - Please refer to [here](https://github.com/MIC-DKFZ/nnUNet) to know how to put all your data in your `${dataset_path}` in right format.
* Our task is to explore reliable kidney semantic segmentation methodologies with the help of labeled AMOS dataset and unlabeled KiTS dataset, evalutaion metric is kidney dice score in target domain.

## 4. Train

- We take [3D-UNet](https://arxiv.org/abs/1606.06650) as users' base model
- We will first pre-train model in AMOS dataset, and use this pre-trained model later for prameter initialization for domain adaptation
- Then we apply domain adaptation algorithm to transfer knowledge from AMOS dataset to KiTS dataset
    - We use a DANN-like model architecture, the DANN algorithm is illustrated as follows:
    ![dann](../imgs/dann.png)
- Notice: 
    - we donot use **any label** from target domain KiTS, we only use label from source domain AMOS for training
    - *For demostration, we only train 1 epochs:*

In [ ]:
! cd modelzoo/unet && sh scripts/run_single_opt.sh 1

+ unset MASTER_ADDR
+ echo ############################## setting env ##############################
############################## setting env ##############################
+ export nnUNet_raw_data_base=/home/vmagent/app/data/adaptor_large/nnUNet_raw_data_base
+ export nnUNet_preprocessed=/home/vmagent/app/data/adaptor_large/nnUNet_preprocessed
+ export RESULTS_FOLDER=/home/vmagent/app/data/adaptor_large/nnUNet_trained_models
+ pre_trained_model_path=/home/vmagent/app/data/adaptor_large/pre-trained-model/model_final_checkpoint-600.model
+ echo ############################## 1 node opt model ##############################
############################## 1 node opt model ##############################
+ epochs=1
+ nnUNet_train_da 3d_fullres nnUNetTrainer_DA_V2 508 507 1 -p nnUNetPlansv2.1_trgSp_kits19 -sp nnUNetPlansv2.1_trgSp_kits19 --epochs 1 --loss_weights 1 0 1 0 0 -pretrained_weights /home/vmagent/app/data/adaptor_large/pre-trained-model/model_final_checkpoint-600.model
scripts/run

## 5. Inference

* We use following command for perform inference and evaluation, you can find your predictions in `${nnUNet_raw_data_base}/nnUNet_raw_data/Task507_KiTS_kidney/predict/`

In [ ]:
! cd modelzoo/unet && sh scripts/run_predict.sh

### Visualization of Data and Segmentations

- For this we would advise to use [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK)) which already has some great [tutorials](https://www.mitk.org/wiki/Tutorials). 
    - If you have not already downloaded it, here is the [MITK Download Link](https://www.mitk.org/wiki/Downloads)
- Here is a demostration of visualization result from MITK on KiTS dataset

![KiTS_visualization](../imgs/KiTS_visualization.png)